In [1]:
import sympy as sym
import numpy as np
from scipy import linalg

In [2]:
mb = 6.
Jxb = 10.
Jyb = 10.
Jzb = 16.

mw = 1.
Jxw = 0.075
Jyw = 0.075
Jzw = 0.125
lw = 1.1

# roll, pitch, yaw angles
phi, theta, psi = sym.symbols('phi, theta, psi')

# angular velocities
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# torques
tau_1, tau_2, tau_3, tau_4 = sym.symbols('tau_1, tau_2, tau_3, tau_4')

# resultant torques
lt = sym.nsimplify(lw) * sym.sqrt(2) / 2
T1 = - tau_1 * sym.Matrix([[lt], [0], [lt]])
T2 = - tau_2 * sym.Matrix([[-lt], [0], [lt]])
T3 = - tau_3 * sym.Matrix([[0], [lt], [lt]])
T4 = - tau_4 * sym.Matrix([[0], [-lt], [lt]])
T = T1 + T2 + T3 + T4

# parameters
Jx = sym.nsimplify(Jxb + 4 * mw * lw**2)
Jy = sym.nsimplify(Jyb + 4 * mw * lw**2)
Jz = sym.nsimplify(Jzb + 4 * mw * lw**2)

# rotation matrices
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])

# angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry * Rz).T * ex, Rz.T * ey, ez).inv(), full=True)

# euler's equations
euler = sym.Matrix([[(1 / Jx) * (T[0] + (Jy - Jz) * w_y * w_z)],
                   [(1 / Jy) * (T[1] + (Jz - Jx) * w_z * w_x)],
                   [(1 / Jz) * (T[2] + (Jx - Jy) * w_x * w_y)]])

# equations of motion
f = sym.simplify(sym.Matrix.vstack(M * sym.Matrix([[w_x], [w_y], [w_z]]), euler), full=True)

In [3]:
f

Matrix([
[                        (w_x*cos(psi) - w_y*sin(psi))/cos(theta)],
[                                     w_x*sin(psi) + w_y*cos(psi)],
[        -w_x*cos(psi)*tan(theta) + w_y*sin(psi)*tan(theta) + w_z],
[-55*sqrt(2)*tau_1/1484 + 55*sqrt(2)*tau_2/1484 - 150*w_y*w_z/371],
[-55*sqrt(2)*tau_3/1484 + 55*sqrt(2)*tau_4/1484 + 150*w_x*w_z/371],
[                -55*sqrt(2)*(tau_1 + tau_2 + tau_3 + tau_4)/2084]])

In [4]:
# Make f an executable function
f_num = sym.lambdify((phi, theta, psi, w_x, w_y, w_z,tau_1, tau_2, tau_3, tau_4),f)

In [5]:
#Equilibrium values

phi_e = 0
theta_e = 0
psi_e = 0
w_x_e = 0
w_y_e = 0
w_z_e = 0
tau_1_e = 0
tau_2_e = 0
tau_3_e = 0
tau_4_e = 0

In [6]:
feq = f_num(phi_e, theta_e
, psi_e
, w_x_e
, w_y_e
, w_z_e
, tau_1_e
, tau_2_e
, tau_3_e
, tau_4_e)

In [7]:
feq #Works perfectly

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [-0.]])

In [8]:
# Now we linearize our system about the equilibrium point
# Linearizing the system

#Now we find the Jacobians
f_jacob_x = f.jacobian([phi, theta, psi, w_x, w_y, w_z])
f_jacob_u = f.jacobian([tau_1, tau_2, tau_3, tau_4])

#And then we find functions for A and B using this jacobian
A_num = sym.lambdify((phi, theta, psi, w_x, w_y, w_z,tau_1, tau_2, tau_3, tau_4),f_jacob_x)
B_num = sym.lambdify((phi, theta, psi, w_x, w_y, w_z,tau_1, tau_2, tau_3, tau_4),f_jacob_u)

#Finally, we find the linearized state space model by evaluating A_num and B_num at the equilibrium points
A = A_num(phi_e, theta_e, psi_e, w_x_e, w_y_e, w_z_e,tau_1_e, tau_2_e, tau_3_e, tau_4_e).astype(float)
B = B_num(phi_e, theta_e, psi_e, w_x_e, w_y_e, w_z_e,tau_1_e, tau_2_e, tau_3_e, tau_4_e).astype(float)

In [9]:
A.tolist()

[[0.0, 0.0, 0.0, 1.0, -0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, -0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, -0.0, -0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [10]:
B.tolist()

[[0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [-0.052413575424878865, 0.052413575424878865, 0.0, 0.0],
 [0.0, 0.0, -0.052413575424878865, 0.052413575424878865],
 [-0.03732329459238015,
  -0.03732329459238015,
  -0.03732329459238015,
  -0.03732329459238015]]

In [11]:
## Checking for controllability
# Find the number of states
n = A.shape[0]

# Initialize W with its first column
W = B

# Create W one column at a time by iterating over i from 1 to n-1
for i in range(1, n):
    col = np.linalg.matrix_power(A, i) @ B
    W = np.block([W, col])

In [12]:
# What about the rank?
print(np.linalg.matrix_rank(W))
#The rank is six, so the system is controllable!

6


In [23]:
# We can directly start designing an LQR controller for the system. It is independent of the observer
Qc = np.diag([40.,20.,20.,30.,25.,25.]) #Roll, pitch, yaw, omegax, omegay, omegaz
Rc = np.diag([1.,1.,1.,1.]) #Tau 1 to tau 4

# We can also define an LQR function in the following way:
def lqr(A,B,Q,R):
    P = linalg.solve_continuous_are(A,B,Q,R)
    K = linalg.inv(R) @ B.T @ P
    return K
# The function above can be used for both controller and observer design
K = lqr(A,B,Qc,Rc)
K.tolist()

[[-4.472135954999583,
  -2.6633170001054275e-15,
  -2.236067977499786,
  -10.016186942499067,
  -3.415143604229942e-15,
  -6.017091808790448],
 [4.472135954999583,
  -2.3056071713205627e-15,
  -2.236067977499785,
  10.016186942499067,
  -2.7307011801662124e-15,
  -6.017091808790448],
 [1.1572690410922678e-15,
  -3.162277660168382,
  -2.2360679774997867,
  -5.759584258290038e-16,
  -8.53423573678893,
  -6.017091808790452],
 [-6.028587380819639e-16,
  3.162277660168377,
  -2.236067977499784,
  1.0848399823472568e-16,
  8.534235736788922,
  -6.017091808790443]]

In [14]:
# Now we work on the Observer
alpha, delta = sym.symbols('alpha, delta')
# Scope radius
r = 0.8 / 2.1
# Position of star in space frame
p_star_in_space = sym.Matrix([[sym.cos(alpha) * sym.cos(delta)],
                              [sym.sin(alpha) * sym.cos(delta)],
                              [sym.sin(delta)]])

# Orientation of body frame in space frame
R_body_in_space = Rx * Ry * Rz

# Position of star in body frame (assuming origin of body and space frames are the same)
p_star_in_body = R_body_in_space.T * p_star_in_space

# Position of star in image frame
p_star_in_image = (1 / sym.nsimplify(r)) * sym.Matrix([[p_star_in_body[1] / p_star_in_body[0]],
                                                       [p_star_in_body[2] / p_star_in_body[0]]])

# Sensor model for each star
g = sym.simplify(p_star_in_image, full=True)

#This is the output equation: y = Cx, where x is the state vector and y is the position of the star given in y-pos and z-pos vars per star
#The following line is taken from the Day30 in-class code

In [15]:
g

Matrix([
[21*((sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))*sin(delta) - (sin(phi)*sin(psi)*sin(theta) - cos(phi)*cos(psi))*sin(alpha)*cos(delta) - sin(psi)*cos(alpha)*cos(delta)*cos(theta))/(8*((sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi))*sin(delta) + (sin(phi)*sin(theta)*cos(psi) + sin(psi)*cos(phi))*sin(alpha)*cos(delta) + cos(alpha)*cos(delta)*cos(psi)*cos(theta)))],
[                                                                      21*(-sin(alpha)*sin(phi)*cos(delta)*cos(theta) + sin(delta)*cos(phi)*cos(theta) + sin(theta)*cos(alpha)*cos(delta))/(8*((sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi))*sin(delta) + (sin(phi)*sin(theta)*cos(psi) + sin(psi)*cos(phi))*sin(alpha)*cos(delta) + cos(alpha)*cos(delta)*cos(psi)*cos(theta)))]])

In [16]:
#Make G a combination of 3 stars with set position data
alpha1 = 0.
alpha2 = 0.15
alpha3 = 0.
alpha4 = -0.2
alpha5 = 0.
delta1 = 0.
delta2 = 0.
delta3 = 0.15
delta4 = 0.
delta5 = -0.2

G = np.block([[g.subs([(alpha, alpha1), (delta, delta1)])],[g.subs([(alpha, alpha2), (delta, delta2)])],[g.subs([(alpha, alpha3), (delta, delta3)])],[g.subs([(alpha, alpha4), (delta, delta4)])],[g.subs([(alpha, alpha5), (delta, delta5)])]])
G = sym.Matrix(G)
G

Matrix([
[                                                                                                                                                                                                                                             -21*sin(psi)/(8*cos(psi))],
[                                                                                                                                                                                                                                 21*sin(theta)/(8*cos(psi)*cos(theta))],
[ 21*(-0.149438132473599*sin(phi)*sin(psi)*sin(theta) - 0.988771077936042*sin(psi)*cos(theta) + 0.149438132473599*cos(phi)*cos(psi))/(8*(0.149438132473599*sin(phi)*sin(theta)*cos(psi) + 0.149438132473599*sin(psi)*cos(phi) + 0.988771077936042*cos(psi)*cos(theta)))],
[                                                         21*(-0.149438132473599*sin(phi)*cos(theta) + 0.988771077936042*sin(theta))/(8*(0.149438132473599*sin(phi)*sin(theta)*cos(psi) + 0.14943

In [17]:
# We also need to find the equilibrium value of this system and subtract that from z to get y
G_num = sym.lambdify((phi, theta, psi),G)
Geq = G_num(phi_e, theta_e, psi_e)
Geq.tolist()

[[-0.0],
 [0.0],
 [0.3967299474030241],
 [0.0],
 [0.0],
 [0.3967299474030241],
 [-0.5321138432102644],
 [0.0],
 [-0.0],
 [-0.5321138432102644]]

In [18]:
#We now create a numerical function for C, that will then be evaluated at the equilibrium point
C_num = sym.lambdify((phi, theta, psi), G.jacobian([phi, theta, psi, w_x, w_y, w_z]))
C = C_num(phi_e, theta_e, psi_e)
C.tolist()

[[0.0, 0.0, -2.625, 0.0, 0.0, 0.0],
 [0.0, 2.625, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, -2.684959867111012, 0.0, 0.0, 0.0],
 [-0.39672994740302475, 2.625, -0.0, 0.0, 0.0, 0.0],
 [0.3967299474030241, 0.0, -2.625, 0.0, 0.0, 0.0],
 [-0.0, 2.684959867111012, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, -2.732864816051809, 0.0, 0.0, 0.0],
 [0.5321138432102653, 2.625, 0.0, 0.0, 0.0, 0.0],
 [-0.5321138432102644, 0.0, -2.625, 0.0, 0.0, 0.0],
 [0.0, 2.732864816051809, -0.0, 0.0, 0.0, 0.0]]

In [19]:
# We can check for the observability of this system
Wo = C
for i in range(1,n):
    row = C @ np.linalg.matrix_power(A,i)
    Wo = np.block([[Wo],[row]])
rank = np.linalg.matrix_rank(Wo)
rank
# The rank of the system is the same as the number of states, so the system is observable.


6

In [20]:
# Designing an optimal controller
Qo = np.diag(2*np.ones(2*5)) # (2n)x(2n) for an n star system
Ro = np.diag([1.,1.,1.,1.,1.,1.])

L = lqr(A.T, C.T, linalg.inv(Ro), linalg.inv(Qo)).T
L.tolist()

[[0.046022430575676704,
  -0.04602243057567631,
  0.04707366822194083,
  -0.7979154886672061,
  0.7979154886672052,
  -0.04707366822194044,
  0.047913554769315754,
  0.9624537569246393,
  -0.9624537569246373,
  -0.04791355476931535],
 [-0.00044463884960331334,
  0.5108369662979904,
  -0.0004547952253498306,
  0.517792576378618,
  -0.0074002489302308905,
  0.5225054297702266,
  -0.00046290966397357974,
  0.5015077577617996,
  0.008884569686587431,
  0.531827951213113],
 [-0.5108369662979906,
  0.00044463884960331334,
  -0.5225054297702267,
  0.007400248930230962,
  -0.5177925763786182,
  0.0004547952253498306,
  -0.5318279512131132,
  -0.008884569686587527,
  -0.5015077577617998,
  0.00046290966397357974],
 [0.024392126532005717,
  -0.024392126532005644,
  0.02494928792835393,
  -0.44874085148696674,
  0.44874085148696613,
  -0.024949287928353853,
  0.025394432147810383,
  0.5447653882340163,
  -0.5447653882340153,
  -0.025394432147810307],
 [-0.00022856549920162683,
  0.441736051714152

In [21]:
#Testing dimensions:
xhat = np.array([[1.],[1.],[1.],[1.],[1.],[1.]])
u = -K@xhat
y = np.array([[1.],[1.],[1.],[1.],[1.],[1.]])
dt = 0.01
K = np.array([[-0.7071067811865472,-1.4320649817540778e-17,-0.5000000000000002,-3.7404423656713486,1.1761215787852632e-15,-2.635949485645772],
 [0.7071067811865474,1.9075460782530832e-16,-0.5000000000000004,3.7404423656713495,1.8131019980329334e-15,-2.635949485645773],
 [-8.784652695069288e-17,-0.7071067811865496,-0.49999999999999983,-4.6119312947957745e-17,-3.740442365671356,-2.6359494856457704],
 [1.582246864518315e-16,0.7071067811865498,-0.5000000000000008,5.908611062997123e-16,3.740442365671359,-2.635949485645775]])
L = np.array([[-0.6265409212125154,0.6265409212125155,-0.6408522775460442,-1.3663913490000046, 1.3663913490000013,  0.6408522775460445], [-0.030420579222051578,0.7171391895243168,-0.03111544165542081,0.6224467907744181,0.06427181952784691,0.7335199782877365], [-0.7171391895243172,0.030420579222051578,-0.733519978287737,-0.06427181952784705,-0.6224467907744188,0.03111544165542081],[-0.2568689610310343, 0.2568689610310346,-0.2627363243713636,-0.6041423348141238,0.6041423348141227,0.262736324371364],[-0.012163724509962885,0.5851185797352603,-0.012441566530988536,0.5462966332974272,0.026658221927870183,0.5984837729867308],[-0.5851185797352606,0.01216372450996261,-0.5984837729867313,-0.026658221927870353,-0.5462966332974277,0.012441566530988254]])
A = np.array([[0.0, 0.0, 0.0, 1.0, -0.0, 0.0],[0.0, 0.0, 0.0, 0.0, 1.0, 0.0],[0.0, 0.0, 0.0, -0.0, 0.0, 1.0],[0.0, 0.0, 0.0, 0.0, -0.0, -0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])
C = np.array([[0.0, 0.0, -2.625, 0.0, 0.0, 0.0],[0.0, 2.625, 0.0, 0.0, 0.0, 0.0],[0.0, 0.0, -2.684959867111012, 0.0, 0.0, 0.0],
 [-0.39672994740302475, 2.625, -0.0, 0.0, 0.0, 0.0],[0.3967299474030241, 0.0, -2.625, 0.0, 0.0, 0.0],[-0.0, 2.684959867111012, 0.0, 0.0, 0.0, 0.0]])
B = np.array([[0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 0.0, 0.0],[0.0, 0.0, 0.0, 0.0],[-0.052413575424878865, 0.052413575424878865, 0.0, 0.0],[0.0, 0.0,-0.052413575424878865,0.052413575424878865],[-0.03732329459238015,-0.03732329459238015,-0.03732329459238015,-0.03732329459238015]])
Geq = np.array([[-0.0], [0.0], [0.3967299474030241], [0.0], [0.0], [0.3967299474030241]])
p = [[0.0, 1.0, 2.0, 3.0, 4.0, 5.0]]
z = np.array(p).T
y = np.array(z - Geq)
#Test += dt * (A @ xhat + B @ u - L @ (C@xhat  - y))
#Test
y.shape

(6, 1)

In [22]:
p = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]
z = np.array([p]).T
z

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.]])